In [39]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


*All the libraries have been downloaded*

In [5]:
from bs4 import BeautifulSoup
import requests

with open('List of postal codes of Canada.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

In [22]:

Postal_code=[]
Borough=[]
Neighbourhood=[]
for row in soup.find('tbody').find_all('tr'):
    cells = row.find_all('td')
    Postal_code.append(cells[0].text.rstrip('\n'))
    Borough.append(cells[1].text.rstrip('\n'))
    try:
        Neighbourhood.append(cells[2].text.rstrip('\n'))
    except:
        Neighbourhood.append('Not Assigned')



Constructing the data frame

In [23]:
import pandas as pd
df = pd.DataFrame({'PostalCode': Postal_code,
                  'Borough': Borough,
                  'Neighborhood': Neighbourhood})
df.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Removing "Not assigned" Boroughs

In [26]:
df1=df[df.Borough !='Not assigned'].reset_index(drop=True)
df1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [27]:
df2= df1.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [36]:
for index,row in df2.iterrows():
    if row["Neighborhood"] == "":
        row["Neighborhood"] = row["Borough"]
        
df2


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [38]:
df2.shape

(103, 3)

Making the dataframe according to the dataframe given

In [42]:
Column_names = ["PostalCode", "Borough", "Neighborhood"]
df3 = pd.DataFrame(columns=Column_names)

list_postal_codes = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in list_postal_codes:
    df3 = df3.append(df2[df2["PostalCode"]==postcode], ignore_index=True)
    
df3

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,Parkview Hill / Woodbine Gardens
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,Wexford / Maryvale
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


In [43]:
df3.to_csv()

',PostalCode,Borough,Neighborhood\n0,M5G,Downtown Toronto,Central Bay Street\n1,M2H,North York,Hillcrest Village\n2,M4B,East York,Parkview Hill / Woodbine Gardens\n3,M1J,Scarborough,Scarborough Village\n4,M4G,East York,Leaside\n5,M4M,East Toronto,Studio District\n6,M1R,Scarborough,Wexford / Maryvale\n7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens\n8,M9L,North York,Humber Summit\n9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport\n10,M1B,Scarborough,Malvern / Rouge\n11,M5A,Downtown Toronto,Regent Park / Harbourfront\n'